# Using `BlackMarblePy`

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldbank/blackmarblepy/blob/main/notebooks/blackmarblepy.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/worldbank/blackmarblepy/main?labpath=notebooks%2Fblackmarblepy.ipynb)

This Jupyter notebook provides a guided exploration of the [**BlackMarblePy**](https://pypi.org/project/blackmarblepy/) package, showcasing its capabilities for downloading, visualizing, and analyzing NASA Black Marble nighttime lights data. Through interactive examples, you'll learn how to:

- Download daily, monthly, and yearly data for specific dates, regions, or bounding boxes.
- Visualize downloaded data in various forms, including maps and  bar charts.
- Save visualizations and analysis results for further use.

## Requirements

Before downloading or extracting [NASA Black Marble data](https://blackmarble.gsfc.nasa.gov/), make sure the following requirements are met:

### 📦 Install `blackmarblepy`

To use this library, install the [blackmarblepy](https://pypi.org/project/blackmarblepy/) package. It is highly recommended to install it inside a [Python virtual environment](https://duckduckgo.com/?t=h_&q=Python+virtual+environment) to avoid dependency conflicts.

#### From PyPI

```shell
pip install blackmarblepy
```

In [ ]:
# Install blackmarblepy and extras for examples
!pip install "blackmarblepy[examples]"

In [ ]:
import os
import datetime

import colorcet as cc
import contextily as cx
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, Title

from blackmarble.extract import bm_extract
from blackmarble.raster import bm_raster

from dotenv import load_dotenv

%load_ext autoreload
%load_ext watermark
%autoreload 2

load_dotenv()
plt.rcParams["figure.figsize"] = (18, 10)

%watermark -v -u -n -p blackmarble

### ✅ Generate and Set Up Your NASA Earthdata Token

**BlackMarblePy** requires a valid, unexpired [NASA Earthdata](https://urs.earthdata.nasa.gov) **bearer token**, which you can retrieve from your [Earthdata profile](https://urs.earthdata.nasa.gov/profile).For ease and security, we recommend [setting this as the BLACKMARBLE_TOKEN environment variable](https://duckduckgo.com/?q=how+to+set+environment+variable+linux+or+mac+or+windows) on your system.

1. Access [Earthdata Login](https://urs.earthdata.nasa.gov/profile). In case you haven't already, you must [register](https://urs.earthdata.nasa.gov/users/new).
    ```{figure} ../images/nasa_earthdata_profile.png
    ---
    height: 150px
    ---
2. Select **Generate Token**. If the token is expired or you are in need of one, click the **Generate Token** button. 
    ```{figure} ../images/nasa_earthdata_generate_token.png
    ---
    height: 400px
    ---
    ```
    
    ```{caution}
    Please be aware that the "Affiliation" information on your [Earthdata profile](https://urs.earthdata.nasa.gov/profile/edit) is mandatory. Without this information, the [NASA Earthdata token](https://urs.earthdata.nasa.gov/documentation) will be invalid. In case of a download error, try visiting the URL which is failing, as you may be prompted to grant permissions.
    
    ![](../images/nasa_earthdata_affiliations.png)
    ```
    
3. Use your token securely with BlackMarblePy. We recommend [setting it as a secret or an environment variable](https://duckduckgo.com/?t=h_&q=environment+variable) rather than hardcoding it. For example, you can set an environment variable to store your token safely:

    ```shell
    export BLACKMARBLE_TOKEN=<your_nasa_earthdata-token>
    ```

    ```{important}
    Using a secret token securely in Python code involves several practices to ensure the token is not exposed unintentionally. For instance, storing the secret token in an environment variable, in configuration files or using secret management services. In this example, we set up an environment variable.
    ```
    
    ```{seealso}
    - [Using Secrets Securely](https://worldbank.github.io/template/notebooks/nasa-apod.html)
    - [Best Practices froor Securing API Keys](https://rapidapi.com/guides/practices-api-keys)
    - [How to use Secrets in Google Colab](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75)
    ```

 ```python
    # An environment variable can obfuscate to secure a secret
    import os
    
    bearer = os.getenv("BLACKMARBLE_TOKEN")
    ```
    
    ```python
    # If using Google Colab, use Secrets
    from google.colab import userdata
    
    bearer = userdata.get('BLACKMARBLE_TOKEN')
    ```

In [ ]:
# Please the item 3 above
# If using Google Colab, you may use Secrets and uncomment below

bearer = os.getenv("BLACKMARBLE_TOKEN")

# If using Google Colab, you may use Secrets and uncomment below
# from google.colab import userdata
# os.environ["BLACKMARBLE_TOKEN"] = userdata.get('BLACKMARBLE_TOKEN')

### 🌍  Define Region of Interest

You must define a region of interest as a [`geopandas.GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html). This should represent the geographic area you want to extract data for.

For example, we obtain the polygon below from [GADM](https://gadm.org/download_country.html) for *Ghana*.

In [ ]:
gdf = geopandas.read_file(
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_GHA_1.json.zip"
)
gdf.explore(tiles="CartoDB dark_matter")

```{figure} ../../images/favicon.ico
---
height: 0px
---
This map of Ghana displays the administrative boundaries as obtained from the Global Administrative Areas (GADM) database. The map highlights the regional divisions and key cities, providing a detailed geographic overview of the country. Data source: GADM, version 4.0.
```

## Examples

In this section, we will demonstrate how to use [BlackMarblePy](https://pypi.org/project/blackmarblepy/) to download and manipulate NASA Black Marble data. 

### Create raster of nighttime lights

In this section, we show examples of creating daily, monthly, and annual rasters of nighttime lights for the **Region of Interest** selected.


#### Daily

In [ ]:
# Daily data: raster for February 5, 2021
VNP46A2_20210205 = bm_raster(
    gdf, product_id="VNP46A2", date_range="2021-02-05", bearer=bearer
)
VNP46A2_20210205

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

VNP46A2_20210205["Gap_Filled_DNB_BRDF-Corrected_NTL"].sel(
    time="2021-02-05"
).plot.pcolormesh(
    ax=ax,
    cmap=cc.cm.bmy,
    robust=True,
)
cx.add_basemap(ax, crs=gdf.crs.to_string())

ax.text(
    0,
    -0.1,
    "Source: NASA Black Marble VNP46A2",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)
ax.set_title("Ghana: NTL Radiance on Feb 5 2021", fontsize=20);

#### Monthly

In [ ]:
# Monthly data: raster for October 2021
VNP46A3_202110 = bm_raster(
    gdf, product_id="VNP46A3", date_range="2021-10-01", bearer=bearer
)
VNP46A3_202110

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

VNP46A3_202110["NearNadir_Composite_Snow_Free"].sel(time="2021-10-01").plot.pcolormesh(
    ax=ax,
    cmap=cc.cm.bmy,
    robust=True,
)
cx.add_basemap(ax, crs=gdf.crs.to_string())

ax.text(
    0,
    -0.1,
    "Source: NASA Black Marble VNP46A3",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)
ax.set_title("Ghana: NTL Radiance in Oct 2021", fontsize=20);

#### Annual

In [ ]:
# Annual data: raster for 2021
VNP46A4_2021 = bm_raster(
    gdf, product_id="VNP46A4", date_range="2021-01-01", bearer=bearer
)
VNP46A4_2021

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

VNP46A4_2021["NearNadir_Composite_Snow_Free"].sel(time="2021-01-01").plot.pcolormesh(
    ax=ax,
    cmap=cc.cm.bmy,
    robust=True,
)
cx.add_basemap(ax, crs=gdf.crs.to_string())

ax.text(
    0,
    -0.1,
    "Source: NASA Black Marble VNP46A4",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)
ax.set_title("Ghana: NTL Radiance in 2021", fontsize=20);

### Create a raster stack of nighttime lights across multiple time periods

In this section, we illustrate how to retrieve and extract [NASA Black Marble](https://blackmarble.gsfc.nasa.gov) data for multiple time periods. The function will return a raster stack, where each raster band corresponds to a different date. The following code snippet provides examples of getting data across multiple days, months, and years. For each example, we define a date range using [`pd.date_range`](https://pandas.pydata.org/docs/reference/api/pandas.date_range.html).

In [ ]:
# Raster stack of daily data
r_daily = bm_raster(
    gdf,
    product_id="VNP46A2",
    date_range=pd.date_range("2022-01-01", "2022-03-31", freq="D"),
    bearer=bearer,
)

In [ ]:
r_daily

In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the mean NTL radiance over the dimensions x and y
r_daily["Gap_Filled_DNB_BRDF-Corrected_NTL"].mean(dim=["x", "y"]).plot(ax=ax)

# Add the data source text
ax.text(
    0,
    -0.2,
    "Source: NASA Black Marble VNP46A2",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)

# Set the title with appropriate fontsize
ax.set_title("Ghana: Daily Average NTL Radiance", fontsize=20, weight="bold")

# Add labels to the axes
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Radiance (nW/cm²/sr)", fontsize=12)

# Adjust layout for better spacing
fig.tight_layout()

# Show the plot
plt.show()

```{figure} ../../images/favicon.ico
---
height: 0px
---
This figures describes the daily average nighttime lights radiance data plotted over time. The data reflects fluctuations in radiance levels due to varying cloud cover, affecting the accuracy of the measurements
```

In [ ]:
# Raster stack of monthly data
r_monthly = bm_raster(
    gdf,
    product_id="VNP46A3",
    date_range=pd.date_range("2022-01-01", "2022-12-31", freq="MS"),
    bearer=bearer,
)

In [ ]:
r_monthly

In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the mean NTL radiance over the dimensions x and y
r_monthly["NearNadir_Composite_Snow_Free"].mean(dim=["x", "y"]).plot(ax=ax)

# Add the data source text
ax.text(
    0,
    -0.2,
    "Source: NASA Black Marble VNP46A3",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)

# Set the title with appropriate fontsize
ax.set_title("Ghana: Monthly Average NTL Radiance", fontsize=20, weight="bold")

# Add labels to the axes
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Radiance (nW/cm²/sr)", fontsize=12)

# Adjust layout for better spacing
fig.tight_layout()

# Show the plot
plt.show()

```{figure} ../../images/favicon.ico
---
height: 0px
---
This figures describes the monthly average nighttime lights radiance data plotted over time. The data reflects fluctuations in radiance levels due to varying cloud cover, affecting the accuracy of the measurements
```

In [ ]:
# Raster stack of yearly data
r_yearly = bm_raster(
    gdf,
    product_id="VNP46A4",
    date_range=pd.date_range("2019-01-01", "2022-01-01", freq="YS"),
    bearer=bearer,
)

In [ ]:
r_yearly

In [ ]:
# Set up the figure and subplots
fig, axs = plt.subplots(2, 2, figsize=(16, 16))

for i, t in enumerate(r_yearly["time"]):
    row = i // 2
    col = i % 2
    ax = axs[row, col]

    r_yearly["NearNadir_Composite_Snow_Free"].sel(time=t).plot.pcolormesh(
        ax=ax,
        cmap=cc.cm.bmw,
        robust=True,
        vmax=50,
    )
    cx.add_basemap(ax, crs=gdf.crs.to_string())

plt.show()

```{figure} ../../images/favicon.ico
---
height: 0px
---
Temporal variation of `NearNadir_Composite_Snow_Free` mapped over multiple years. Each subplot represents a different time period overlaid with a basemap.
```

### Downloading and Saving Black Marble Data Locally

In this section, we provide a guide on using [BlackMarblePy](https://worldbank.github.io/blackmarblepy) to download [NASA Black Marble](https://blackmarble.gsfc.nasa.gov) data and save it to a specified local directory. You can use the `output_directory` parameter to designate the directory for saving the files. By default, files that have already been downloaded will not be re-downloaded in subsequent executions.

In [ ]:
bm_raster(
    gdf,
    product_id="VNP46A4",
    date_range=pd.date_range("2022-01-01", "2022-01-01", freq="YS"),
    output_directory="data/",
    output_skip_if_exists=True,  # default
    bearer=bearer,
)

Alternatively, set `output_skip_if_exists=False` to force the redownload of files from NASA.

In [ ]:
bm_raster(
    gdf,
    product_id="VNP46A4",
    date_range=pd.date_range("2022-01-01", "2022-01-01", freq="YS"),
    output_directory="data/",
    output_skip_if_exists=False,
    bearer=bearer,
)

### Visualizing difference in radiance year over year

Lastly, we calculate the increase/decrease in nighttime lights radiance levels.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

delta = (
    (
        (
            r_yearly["NearNadir_Composite_Snow_Free"].sel(time="2022-01-01")
            - r_yearly["NearNadir_Composite_Snow_Free"].sel(time="2019-01-01")
        )
        / r_yearly["NearNadir_Composite_Snow_Free"].sel(time="2019-01-01")
    )
    # .drop("time")
    .plot.pcolormesh(ax=ax, cmap="Spectral", robust=True)
)
cx.add_basemap(ax, crs=gdf.crs.to_string(), source=cx.providers.CartoDB.DarkMatter)

ax.text(
    0,
    -0.1,
    "Source: NASA Black Marble VNP46A3",
    ha="left",
    va="center",
    transform=ax.transAxes,
    fontsize=10,
    color="black",
    weight="normal",
)
ax.set_title("Ghana: NTL Radiance Increase/Decrease (2019-2022)", fontsize=16);

```{figure} ../../images/favicon.ico
---
height: 0px
---
This figure displays the percentage change in radiance for Ghana between 2019 and 2022. The data, sourced from NASA's Black Marble VNP46A3 product is visualized with basemap.
```

### Compute trends on nighttime lights over time

In this section, we use the `bm_extract` function to observe treends in nighttime lights over time. The `bm_extract` function leverages the [rasterstats](https://pythonhosted.org/rasterstats/) package to aggregate nighttime lights data to polygons. In the following example, we show trends in annual nighttime lights data across Ghana's first-administrative divisions.

In [ ]:
VNP46A4 = bm_extract(
    gdf,
    "VNP46A4",
    pd.date_range("2012-01-01", "2023-01-01", freq="YS"),
    bearer,
    output_directory="data/",
)

In [ ]:
p = figure(
    title="Ghana Regions: Annual Average Nighttime Lights Radiance",
    width=760,
    height=600,
    x_axis_label="Year",
    x_axis_type="datetime",
    y_axis_type="log",
    y_axis_label="NTL Radiance",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
)
p.add_layout(
    Title(
        text=f"Source: NASA Black Marble. Creation date: {datetime.datetime.today().strftime('%d %B %Y')}.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_tools(
    HoverTool(
        tooltips=[
            ("Year", "@x{%Y}"),
            ("Radiance", "@y{0.00}"),
        ],
        formatters={"@x": "datetime"},
    )
)

data = VNP46A4.pivot_table(
    index="date", columns=["NAME_1"], values=["ntl_mean"], aggfunc="mean"
)

for column, color in zip(data.columns, cc.glasbey):
    r = p.line(
        data.index,
        data[column],
        legend_label=column[1],
        line_color=color,
        line_dash="dotdash",
        line_width=2,
    )

p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"

p.add_layout(p.legend[0], "right")

output_notebook()
show(p)

```{figure} ../../images/favicon.ico
---
height: 0px
---
This figure illustrates the trend in nighttime lights radiance across Ghana over the period from 2012 to 2023. The X-axis represents the years from 2012 to 2023, while the Y-axis measures the radiance in nanowatts per square centimeter per steradian (nW/cm²/sr). The data highlights changes in light intensity, which can be indicative of economic activity, urbanization, and infrastructure development. Data source: NASA Black Marble VNP46A4 retrieved with [BlackMarblePy](https://worldbank.github.io/blackmarblepy).
```

## References 

{cite:empty}`ROMAN2018113`
{cite:empty}`blackmarblepy`

```{bibliography}
:filter: docname in docnames
:style: plain
```